In [57]:
import random
from pprint import pprint

# Parâmetros do AG
POPULATION_SIZE = 20
MUTATION_RATE = 0.03
CROSSING_RATE = 0.8
MAX_GENERATIONS = 1000

# Gera um board state aleatório
def generate_board_state():
    board_state = [format(random.randint(0, 7), '03b') for _ in range(8)]
    return board_state

# Calcula o fitness do board_state
def calculate_fitness(board_state):
    conflicts = 0
    for i in range(8):
        for j in range(i + 1, 8):
            # Para identificar conflitos nas diagonais:
            # a diferença entre as linhas é igual a diferença entre as colunas
            if int(board_state[i], 2) == int(board_state[j], 2) or abs(int(board_state[i], 2) - int(board_state[j], 2)) == j - i:
                conflicts += 1
    return 28 - conflicts  # fitness máximo = 28 (sem conflitos)

# Seleciona os pais para o crossover usando a estratégia da roleta
def roullete_selection(population):
    total_fitness = sum(individual[1] for individual in population)
    pick = random.uniform(0, total_fitness)
    current = 0
    for individual in population:
        current += individual[1]
        if(current >= pick):
            return individual

# Operação de crossover (estratégia do ponto de corte)
def crossover(parent1, parent2):
    crossover_point = random.randint(1, 7)
    child = parent1[:crossover_point] + parent2[crossover_point:]
    return child

# Operação de mutação (estratégia do bit flip)
def mutate(board_state):
    pos = random.randint(0, 7) # posicao aleatoria do tabuleiro para fazer o flip
    bit_to_flip = random.randint(0,2) # escolhe um dos 3 bits para flipar
    flipped_bit = '1' if board_state[pos][bit_to_flip] == '0' else '0'
    new_bin_pos = board_state[pos][:bit_to_flip] + flipped_bit + board_state[pos][bit_to_flip+1:]
    board_state[pos] = new_bin_pos
    return board_state

# Gera população inicial
population = [(generate_board_state(), 0) for _ in range(POPULATION_SIZE)]
# Loop principal do AG
for generation in range(MAX_GENERATIONS):
    # Calcula fitness para cada board_state
    population = [(board_state, calculate_fitness(board_state)) for board_state, _ in population]
    # Verifica se a solução foi encontrada
    best_board_state = max(population, key=lambda x: x[1])[0]
    if calculate_fitness(best_board_state) == 28:
        print("\033[1;32mSolução encontrada na geração:", generation, "\033[0;39m")
        break

    # Cria próxima geração
    new_population = []

    # Elitista: Mantém o melhor board_state da geração
    new_population.append(max(population, key=lambda x: x[1]))

    # Executa seleção, crossover e mutação
    while len(new_population) < POPULATION_SIZE:
        parent1 = roullete_selection(population)
        parent2 = roullete_selection(population)
        if random.random() < CROSSING_RATE:
            child = crossover(parent1[0], parent2[0])
            if random.random() < MUTATION_RATE:
                child = mutate(child)
            new_population.append((child, 0))
        else:
            new_population.append(parent1)
            new_population.append(parent2)

    # Atualiza a população
    population = new_population

# Printa a melhor solução
print("Melhor Solução (codificação binária): ", best_board_state)
boardStateInt = [int(binary,2) for binary in best_board_state]
print("Melhor Solução (codificação inteira): ", boardStateInt)
print("fitness: ", calculate_fitness(best_board_state))


Solução encontrada na geração: 150 
Melhor Solução (codificação binária):  ['101', '010', '110', '001', '111', '100', '000', '011']
Melhor Solução (codificação inteira):  [5, 2, 6, 1, 7, 4, 0, 3]
fitness:  28
